In [3]:
import pandas as pd
import numpy as np

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset= pd.get_dummies(dataset, dtype = int, drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [7]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [8]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[["charges"]]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(independent,dependent, test_size=0.30, random_state=0)
x_train,x_test,y_train,y_test

(      age     bmi  children  sex_male  smoker_yes
 1163   18  28.215         0         0           0
 196    39  32.800         0         0           0
 438    52  46.750         5         0           0
 183    44  26.410         0         0           0
 1298   33  27.455         2         1           0
 ...   ...     ...       ...       ...         ...
 763    27  26.030         0         1           0
 835    42  35.970         2         1           0
 1216   40  25.080         0         1           0
 559    19  35.530         0         1           0
 684    33  18.500         1         0           0
 
 [936 rows x 5 columns],
       age     bmi  children  sex_male  smoker_yes
 578    52  30.200         1         1           0
 610    47  29.370         1         0           0
 569    48  40.565         2         1           1
 1034   61  38.380         0         1           0
 198    51  18.050         0         0           0
 ...   ...     ...       ...       ...         ...
 126

In [20]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
x_test


array([[ 0.89459283, -0.08863026, -0.06746417,  1.02378711, -0.50466988],
       [ 0.53757957, -0.22180837, -0.06746417, -0.97676557, -0.50466988],
       [ 0.60898222,  1.57449152,  0.76341038,  1.02378711,  1.98149332],
       ...,
       [ 1.10880078,  1.20785059, -0.89833872,  1.02378711, -0.50466988],
       [ 1.75142463,  1.34905148, -0.06746417,  1.02378711, -0.50466988],
       [ 1.60861933, -0.92299913, -0.89833872, -0.97676557, -0.50466988]])

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
param_grid= {'criterion':["squared_error", "friedman_mse", "absolute_error", "poisson"],'splitter':['best','random'], 'max_features':['sqrt', 'log2',None]}
grid=GridSearchCV(DecisionTreeRegressor(),param_grid,refit=True, verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             verbose=3)

In [34]:
grid_prediction=grid.predict(x_test)

0.7498770528520761

In [36]:
re=grid.cv_results_
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_prediction)

print("The R_score value for best parameter {}".format(grid.best_params_),r_score)

The R_score value for best parameter {'criterion': 'squared_error', 'max_features': 'sqrt', 'splitter': 'best'} 0.7498770528520761


In [19]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011699,0.001683,0.006188,0.001869,squared_error,sqrt,best,"{'criterion': 'squared_error', 'max_features':...",0.780404,0.695153,0.646100,0.674803,0.670226,0.693337,0.046245,1
1,0.010692,0.005860,0.008923,0.002843,squared_error,sqrt,random,"{'criterion': 'squared_error', 'max_features':...",0.704248,0.561721,0.616067,0.688830,0.610720,0.636317,0.052919,11
2,0.010727,0.002938,0.003274,0.002777,squared_error,log2,best,"{'criterion': 'squared_error', 'max_features':...",0.645248,0.547753,0.674267,0.724359,0.662301,0.650786,0.057868,6
3,0.006357,0.002546,0.003230,0.004014,squared_error,log2,random,"{'criterion': 'squared_error', 'max_features':...",0.481020,0.636387,0.637369,0.663038,0.678621,0.619287,0.070953,15
4,0.012807,0.004023,0.005542,0.001275,squared_error,None,best,"{'criterion': 'squared_error', 'max_features':...",0.741122,0.573054,0.772426,0.620733,0.648535,0.671174,0.074604,2
5,0.007961,0.001681,0.004862,0.002548,squared_error,None,random,"{'criterion': 'squared_error', 'max_features':...",0.623991,0.663796,0.695982,0.546970,0.537057,0.613559,0.062790,18
6,0.009389,0.003683,0.006647,0.001750,friedman_mse,sqrt,best,"{'criterion': 'friedman_mse', 'max_features': ...",0.714451,0.493134,0.686977,0.587409,0.537513,0.603897,0.084937,19
7,0.008014,0.002566,0.005255,0.003029,friedman_mse,sqrt,random,"{'criterion': 'friedman_mse', 'max_features': ...",0.751276,0.558026,0.580770,0.476488,0.570202,0.587352,0.089844,21
8,0.009398,0.000436,0.004063,0.001091,friedman_mse,log2,best,"{'criterion': 'friedman_mse', 'max_features': ...",0.530900,0.610863,0.652712,0.636352,0.655996,0.617365,0.046089,17
9,0.009766,0.002453,0.004166,0.003886,friedman_mse,log2,random,"{'criterion': 'friedman_mse', 'max_features': ...",0.713957,0.528668,0.568189,0.590466,0.461333,0.572523,0.083223,23


In [32]:
age_input=float(input("Age:"))
bmi_input=float(input('Bmi'))
children=float(input("Children"))
sex_male=int(input("sex male 0 or 1"))
smoker_yes=int(input("Smoker yes 0 or 1"))

Age: 4
Bmi 56
Children 5
sex male 0 or 1 5
Smoker yes 0 or 1 3


In [33]:
Future_Prediction=grid.predict([[age_input,bmi_input,children,sex_male,smoker_yes]])
print("Future_Prediction:{}".format(Future_Prediction))

Future_Prediction:[46130.5265]
